In [2]:
import os , os.path #osを読み込む
import sys
import pandas as pd #xlsxのファイルをcsvにする
import ast
import requests
from bs4 import BeautifulSoup
import json
from urllib import request
import csv
import collections
import glob
import re

In [3]:
#シリーズごとに製品を絞り込んでいる
df3 = pd.read_excel("カメラ辞書.xlsx", sheet_name=0)
df3.to_csv("カメラ辞書.csv", index=False, header=None, encoding='utf_8_sig')
with open("カメラ辞書.csv", encoding='utf_8_sig') as fp:
    camera_query = df3.query('シリーズ == "EOS" & 種別 == "ミラーレス一眼"')
    siborikomi = camera_query["品名"]#絞り込んだ結果の品名を取り出す”EOS R5_body”とか

In [4]:
selected_products = []#絞り込んだ結果をリストにする
for selects in siborikomi:
    selected_products.append(selects)
print(selected_products)

['EOS R5_body']


In [5]:
#評価表現辞書読み込み
df1 = pd.read_excel("評価表現辞書.xlsx", sheet_name=0)
df1.to_csv("評価表現辞書.csv", index=False, header=None, encoding='utf_8_sig')
with open("評価表現辞書.csv", encoding='utf_8_sig') as fp:
    
    #tuple型の評価表現辞書作成
    all_list = [] #all_lstはtuple型の評価表現辞書全部
    reader = csv.reader(fp)
    for row in reader: #rowはキヤノンらしいのかたまり
        word_list = [] #elementを追加していったもの
        for element in row: #elementはキヤノンらしいなどの単語1つ1つ
            if element != '': #空白削除
                word_list.append(element) #word_lstに追加していく
                word_tuple = tuple(word_list) #word_tupleはword_lstをタプル型に変換
        #print(word_tuple[0])
        all_list.append(word_tuple) #all_lstに追加していく
    #print(all_lst[0])


#all_lstをキーに0を値に取るdict型作成
mydict = {} #mydictはall_lstをキーに0を値に取る
for word in all_list:
    mydict[word] = 0
#print(mydict)

for series in selected_products:#品名ごとに抜き出す
    #csvファイル読みこむ
    with open("./review/{}.csv".format(series)) as fp:
        reader = csv.reader(fp)
        for row in reader:
            review = row[4]
            
            #mydictの値の部分をカウントしていく
            skip_flag = False #Falseは0と同じ
            for tuple_list in mydict: #tuple_listはキヤノンらしいのかたまり
                for item in tuple_list: #itemはキヤノンらしいなどの単語1つ1つ
                    if review.count(item) > 0: #数えた結果が0より大きかったら
                        skip_flag = True #Trueは1と同じ
                        continue #次のitemに行く

                if skip_flag == True: #skip_flagがTrueなら処理実行
                    skip_flag = False #Trueを踏んだらFalseに戻す
                    mydict[tuple_list] += 1 #mydictの値のカウントを１増やす
                    continue #次のtuple_listに行く 
   
    print(mydict)


{('携帯性が良い', '携帯性が良く', '携帯性はいい', '携帯性は良い', '携帯性は良かった', '携帯性もいい', '携帯性に優れ', '携帯性も優れ', '携帯性抜群', '携帯性は抜群', '携帯性はバツグン', '携帯性は高い', '携帯性は充分', '携帯性は悪くない', '携帯性はピカイチ', '携帯しやすい'): 0, ('持ち運びやすい', '持ち運び易い', '持ち運びしやすい', '持ち運びのしやすさ', '持ち運びが楽', '持ち歩きやすい'): 0, ('携帯性は悪い', '携帯性は悪く', '携帯性はイマイチ', '携帯性が落ちる', '携帯性が難あり'): 0, ('操作性が良い', '操作性が良く', '操作性はいい', '操作性も良い', '操作性もいい', '操作が容易', '操作は容易', '操作しやすい', '操作はしやすい', '操作しやすく', '操作し易い', '操作はし易い', '操作性の向上', '操作性は抜群', '操作感が好き', '簡単に操作できる', '簡単に操作でき'): 21, ('操作性が悪い', '操作性が悪く', '操作性も悪く', '操作しにくい', '操作がしにくい', '操作系は最悪', '操作性が駄目', '操作性はイマイチ'): 0, ('操作性の物足りなさ',): 0, ('手ぶれ補正がある', '手振れ補正がある', '手振れ補正もある', '手振れ補正あり', '手振れ補正もあり', '手振れ補正が搭載', '手ぶれ補正も付いている', '手振れ補正内蔵', '手振れ補正', '手ブレ補正', '高性能な手ブレ防止'): 29, ('手振れ機能は良い', '手振れ補正が素晴らしい', '手ぶれ補正は素晴らしい'): 0, ('手ぶれ補正が強力', '手ブレ補正が強力', '手ブレ補正が強化', '手振れ補正が強力', '手ぶれ補正がきき', '手ぶれ補正が効き', '手ぶれ補正は効いて', '手ブレ補正は効いて', '手ブレ補正も効いて', '手振れ補正は効いて', '手ぶれ補正が効く'): 0, ('手ぶれ補正がない', '手ブレ補正がない', '手ブレ補正のない', '手振れ補正がない', '手振れ補正はない', '手振れ補正が無い', '手振れ補正は無い', '手ぶれ補